결측값때문에 행의 개수가 달라짐 -> 결측값 처리가 우선시 되어야 함 -> interpolate 를 통해 선형 보간 해줌

In [1]:
# from . import _distributor_init
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
ulsan_fcst = pd.read_csv('data/ulsan_fcst_data.csv') # 울산시 기상예보 데이터
ulsan_obs = pd.read_csv('data/ulsan_obs_data.csv') # 울산시 기상관측 데이터
energy = pd.read_csv('data/energy.csv') # 에너지 발생량 데이터 (여기에서는 ulsan 컬럼의 데이터를 사용)

In [3]:
ulsan_obs_df = ulsan_obs.copy()
ulsan_fcst_df = ulsan_fcst.copy()

In [4]:
ulsan_fcst_df = ulsan_fcst_df.interpolate()

In [5]:
ulsan_obs_df.describe()

,지점,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위)
count,25632.0,25628.000000,25631.000000,25631.000000,25631.000000,24807.000000
mean,152.0,15.008627,2.135761,232.934337,67.328743,5.067441
std,0.0,8.771188,1.223418,121.994098,23.206865,3.999174
min,152.0,-12.000000,0.000000,0.000000,7.000000,0.000000
25%,152.0,8.200000,1.200000,140.000000,50.000000,0.000000
50%,152.0,15.700000,2.000000,290.000000,70.000000,6.000000
75%,152.0,21.800000,2.900000,340.000000,88.000000,9.000000
max,152.0,36.500000,11.100000,360.000000,98.000000,10.000000


In [6]:
ulsan_obs_df = ulsan_obs_df.interpolate()

In [7]:
# ulsan_obs[ulsan_obs['일시'].dt.hour==14]
ulsan_obs_df['일시'] = pd.to_datetime(ulsan_obs['일시'])

In [8]:
# 예보 시간 컬럼의 데이터 타입을 datetime으로 변경함
ulsan_fcst_df['Forecast_time'] = pd.to_datetime(ulsan_fcst_df['Forecast time'])

In [9]:
# 기상청에서는 하루 3시간 간격으로 8회 예보함
# 여기서는 14시 예보만을 사용함
fcst_14 = ulsan_fcst_df[ulsan_fcst_df['Forecast_time'].dt.hour==14]

In [10]:
# 다음 날의 기상 예보가 필요하여 예보시간 기준 10시간 후(00:00)부터 33시간 후(23:00) 데이터만 사용함
fcst_14 = fcst_14[(fcst_14['forecast']>=10)&(fcst_14['forecast']<=33)]

In [11]:
# 예보 시점에 focast를 더하여 예보 시각을 구함
def to_date(x):
    return pd.DateOffset(hours=x)

fcst_14['Forecast_time'] = fcst_14['Forecast_time'] + fcst_14['forecast'].map(to_date)

D:\Anaconda\envs\data_engeneering\lib\site-packages\pandas\core\arrays\datetimelike.py:1111: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,


In [12]:
fcst_14 = fcst_14[['Forecast_time', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']]

In [13]:
# 태양광 발전량 예측은 1시간 간격으로 해야하지만 예보는 3시간 간격으로 나옴
# 간단한 선형보간법을 활용하여 비어있는 값을 채움
# 우선 한시간 간격의 데이터프레임을 생성함
fcst_14_ = pd.DataFrame()
fcst_14_['Forecast_time'] = pd.date_range(start='2018-03-02 00:00:00', end='2021-03-01 23:00:00', freq='H')

In [14]:
# 기존 예보 데이터프레임과 병합함
fcst_14_ = pd.merge(fcst_14_, fcst_14, on='Forecast_time', how='outer')

In [15]:
inter_fcst_14 = fcst_14_.interpolate() # interpolate 를 통해 선형보간해줌

In [16]:
energy_df = energy.copy()

In [17]:
energy_df = energy_df.interpolate()

In [18]:
temp_ulsan_obs_df = ulsan_obs_df[24:].reset_index(drop=True) # forecast 데이터를 3월 2일부터 사용하기 때문에 obs 데이터에서도 하루를 생략
temp_ulsan_obs_df['일시'] = pd.to_datetime(temp_ulsan_obs_df['일시']) # 일시열의 데이터를 object 타입에서 to_datetime 타입으로 변경
inter_fcst_14.rename(columns={'Forecast_time' : '일시'}, inplace=True) # Forecast_time열의 이름을 일시로 변경
drop_inter_fcst_14 = inter_fcst_14.drop_duplicates('일시').reset_index(drop=True) # 일시열의 중복된 행 중 첫번째만 남기고 제거
ulsan_obs_fcst_df = pd.merge(temp_ulsan_obs_df, drop_inter_fcst_14, on='일시') # temp_dangjin_obs_df 와 drop_inter_fcst_14 를 '일시' 열을 기준으로 합병

In [19]:
temp_energy_df = energy_df['ulsan'][23:].copy().reset_index(drop=True)
ulsan_obs_fcst_energy_df = pd.merge(ulsan_obs_fcst_df, temp_energy_df, left_index=True, right_index=True) # dangjin_obs_fcst_df 와 energy_df['dangjin'] 을 합병

In [20]:
# date time 사용법
date_time = pd.to_datetime(ulsan_obs_fcst_energy_df['일시'], format='%Y-%m-%d %H:%M')

In [21]:
ulsan_obs_fcst_energy_df['log_dangjin'] = ulsan_obs_fcst_energy_df['ulsan'].apply(np.log1p)

In [22]:
ulsan_obs_fcst_energy_df['month'] = ulsan_obs_fcst_energy_df['일시'].dt.month # 월 값으로 month 열 데이터를 만들어줌
ulsan_obs_fcst_energy_df['hour'] = ulsan_obs_fcst_energy_df['일시'].dt.hour # 시간 값으로 hour 열 데이터를 만들어줌

In [23]:
ex1 = ulsan_obs_fcst_energy_df.copy()

In [24]:
ex1 = ulsan_obs_fcst_energy_df.drop(['지점', '지점명', 'log_dangjin'], axis=1)

In [25]:
ex1.rename(columns = {'일시' : 'date'}, inplace = True)

In [26]:
ex1[50:100]

,date,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),Temperature,Humidity,WindSpeed,WindDirection,Cloud,ulsan,month,hour
50,2018-03-04 02:00:00,9.2,1.4,360.0,83.0,0.666667,9.333333,85.000000,0.966667,280.333333,3.000000,0,3,2
51,2018-03-04 03:00:00,10.2,2.8,320.0,82.0,1.000000,9.000000,85.000000,0.900000,288.000000,3.000000,0,3,3
52,2018-03-04 04:00:00,8.8,1.1,360.0,85.0,1.000000,8.666667,86.666667,1.133333,299.333333,3.000000,0,3,4
53,2018-03-04 05:00:00,7.8,0.5,360.0,88.0,0.000000,8.333333,88.333333,1.366667,310.666667,3.000000,0,3,5
54,2018-03-04 06:00:00,6.7,0.8,140.0,95.0,0.000000,8.000000,90.000000,1.600000,322.000000,3.000000,0,3,6
55,2018-03-04 07:00:00,7.2,0.7,20.0,98.0,0.000000,9.000000,86.666667,1.400000,332.666667,3.000000,0,3,7
56,2018-03-04 08:00:00,11.3,0.9,320.0,89.0,2.000000,10.000000,83.333333,1.200000,343.333333,3.000000,20,3,8
57,2018-03-04 09:00:00,13.0,1.2,360.0,73.0,1.000000,11.000000,80.000000,1.000000,354.000000,3.000000,91,3,9
58,2018-03-04 10:00:00,16.2,1.2,140.0,62.0,3.000000,12.666667,76.666667,1.066667,291.333333,3.000000,166,3,10
59,2018-03-04 11:00:00,17.1,2.0,160.0,68.0,6.000000,14.333333,73.333333,1.133333,228.666667,3.000000,257,3,11


In [28]:
ex1.to_csv('data/ulsan_obs_fcst_energy_df.csv', index=None)